# Parte A

## Sistema de paragem ABS


1. Defina um autómato híbrido que descreva a dinâmica do sistema de travagem ABS. Modele em lógica temporal LT  propriedades que, na sua opinião, caracterizam o comportamento desejável do sistema.
2. Codifique em SMT’s o modelo que definiu em 1.
3. Codifique a verificação das propriedades temporais que definiu em 1 .

In [70]:
from z3 import *
import sys, os

In [71]:
Modes, (Start, Free, Stopping, Blocked, Stopped) = EnumSort('Modes',['Start', 'Free', 'Stopping', 'Blocked', 'Stopped'])
Events, (start, goStopping, goFree, goBlocked, goStopped , alive , dead) =  \
    EnumSort('Events',['start', 'goStopping', 'goFree', 'goBlocked', 'goStopped' , 'alive' , 'dead'])
    


In [72]:
N = 10
Fd = 0.1
Fe = 1
VP = 2
a = 0.5
b = 1
c = 1
vv   = RealVector('vv', N+1) #velocidade do veiculo
vr   = RealVector('vr', N+1) #velocidade das rodas
temp = RealVector('t', N+1) #velocidade das rodas
modes  = AstVector() ; modes.resize(N+1)
events = AstVector() ; events.resize(N+1)

for k in range(N+1):
    modes[k] = Const('md_'+str(k), Modes)

for k in range(N+1):
    events[k] = Const('ev_'+str(k), Events)

def S(k):
    return (vv[k],vr[k],temp[k],modes[k])

def E(k):
    return events[k]

In [73]:
def init(S):
    (V,R,T,M) = S 
    return And(R <= V, M == Start)

In [74]:
def inv(S):
    (V,R,T,M) = S 
    return And(Implies( M == Start    , T == 0              ),
               Implies( M == Free     , V >= R              ),
               Implies( M == Stopping , And(R >= 0, V >= R) ),
               Implies( M == Blocked  , And(R == 0, V >= 0) ),
               Implies( M == Stopped  , And(V == R, R == 0) )
             )

In [75]:
def timed(S,S_,E):
    (V,R,T,M) = S ; (V_,R_,T_,M_) = S_ 
    return And( E == alive, 
                M == M_,
                T_ > T,
                Implies( M == Start    , And(V == V_, R == R_)                                        ),
                Implies( M == Free     , And((V_ - V) + Fd*(T_ - T) == 0, (R_ - R) - c*(T_ - T) == 0) ),
                Implies( M == Stopping , And((V_ - V) + a*(T_ - T) == 0, (R_ - R) + b*(T_ - T) == 0)  ),
                Implies( M == Blocked  , And((V_ - V) + Fe*(T_ - T) == 0, R == R_, R == 0)            ),
                Implies( M == Stopped  , And(V == V_, R == R_)                                        )
              ) 

In [76]:
#-----------------------
def untimed(S,S_,E):
    (V,R,T,M) = S ; (V_,R_,T_,M_) = S_ 
    return And( V == V_,
                R == R_,
                T == T_,
                E != dead,
                E != alive,
                Implies( E == start      , And( M == Start    , M_ == Free     , R <= V      )),
                Implies( E == goStopping , And( M == Free     , M_ == Stopping , V - R == 0  )),
                Implies( E == goFree     , And( M == Stopping , M_ == Free     , V - R >= VP )),
                Implies( E == goBlocked  , And( M == Stopping , M_ == Blocked  , R == 0      )),
                Implies( E == goStopped  , And( M == Blocked  , M_ == Stopped  , V == 0      ))
              )

In [77]:
def blocked(S,S_,E): 
    (V,R,T,M) = S ; (V_,R_,T_,M_) = S_ 
    return  And(E == dead, V_ == V, R_ == R, T_ == T, M_ == M)

In [78]:
def trans(S,S_,E): 
    return Or(blocked(S,S_,E), timed(S,S_,E), untimed(S,S_,E))

In [79]:
s = Solver()

s.add(init(S(0)))

for k in range(N-1):
    s.add(inv(S(k)))
    s.add(trans(S(k),S(k+1),E(k)))  

s.add(inv(S(N)))

print(s.check())

sat


In [80]:


try:
    M = s.model()
    for z in range(N-1):
        print(M[temp[z]], M[vv[z]], M[vr[z]], M[modes[z]])
        print('--')
        print(M[events[z]])
        print('-->')
except Z3Exception as ex:
    print("failed: %s" % ex)

0 13/3 7/3 Start
--
start
-->
0 13/3 7/3 Free
--
alive
-->
5/33 95/22 82/33 Free
--
alive
-->
10/33 142/33 29/11 Free
--
alive
-->
20/11 137/33 137/33 Free
--
goStopping
-->
20/11 137/33 137/33 Stopping
--
alive
-->
64/11 71/33 5/33 Stopping
--
goFree
-->
64/11 71/33 5/33 Free
--
dead
-->
64/11 71/33 5/33 Free
--
alive
-->


# Verificação de propriedades

Verifique as possibilidades de:  

* existencia de situacao onde as rodas andam mais que o veiculo;
* existencia de situacao onde as rodas andam mais que o 10 de velocidade;





In [81]:
def P1(S):
    (V,R,T,M) = S
    return R>V 

def P2(S):
    (V,R,T,M) = S
    return R>10 


In [82]:
s = Solver()

s.add(init(S(0)))

for k in range(N-1):
    s.add(P1(S(k)))
    s.add(trans(S(k),S(k+1),E(k)))  

s.add(P1(S(N)))

print(s.check())

unsat


In [83]:
try:
    M = s.model()
    for z in range(N-1):
        print(M[temp[z]], M[time[z]], M[modes[z]])
        print('--')
        print(M[events[z]])
        print('-->')
except Z3Exception as ex:
    print("failed: %s" % ex)

failed: model is not available


# Parte B

1. Codifique o programa na linguagem de comandos anotados apresentada nas aulas teóricas.
2. Codifique, em SMT’s, a verificação da correcção parcial.

## Codificar o algoritmo de Euclides da questao 2 do Trabalho 2

0: $assert\quad a > 0\quad and\quad b > 0$  
1: $while\quad a > 0\quad and\quad b > 0:$  
2: $\hspace{1cm}if\quad b > a$  
3: $\hspace{2cm}b = b - a$  
4: $\hspace{1cm}else:$  
5: $\hspace{2cm}a = a - b$    
6: $pass$ 


## Código em 'comandos anotados'

```
# pré-condição:
assume a > 0  and  b > 0;
x := a;
y := b;

# condição para a entrada no ciclo:
assert INV(x,y);

# condição para cada iteracção: 
havoc x;
havoc y;
assume INV(x,y);


if x = 0:
    # condição para a saída do ciclo (pós-condição)
    assert INV(x,y) and (y = gcd(a,b));
else:
    if y > x:
        y := y - x
    else:
        x := x - y
    ;
    # condição para a saída de iteracção:
    assert INV(x,y)
```

###  Invariante

```
INV(x,y)  define-se como 
    x>=0 and y>0
```

### Semântica da função 'gcd' expressa na tautologia  GCD


```
GCD <==> forall x,y : INV(x,y) and (x equiv y)  ==>  y_3 = gcd(a,b)

(x equiv y)  <==>  x_3 % y_3 = 0    
```

## INV:
O nosso invariante será:

In [84]:
def INV(x,y):
    return And(x>=0, y>0) 

In [85]:
gcd = Function('gcd',IntSort(),IntSort(),IntSort())

In [86]:
def GCD():   
    A = nounce('X', IntSort());  B = nounce('Y', IntSort())   
    return ForAll([A,B],Implies(
        And(INV(A,B), z(A) % z(B) == 0), 
        z(B) == gcd(a,b)))

In [87]:
a = Int(27)
b = Int(6)

import random 

def nounce(pref,srt=BoolSort()):
    n = hex(hash(random.random()))[-6:-2]   # 
    return Const(pref + n, srt)


X = nounce('X',IntSort())
Y = nounce('Y',IntSort())


def spc(f):
    return simplify(B2(B1(B0(f))))


    
def B0(f):
    A = nounce('A',IntSort()) ;  B = nounce('B',IntSort())
    F = And(a > 0 , b > 0, f)
    Q = And(substitute(F,(X,A),(Y,B)),X==a,Y==b)
    return Implies(Q,INV(X,Y))

def B1(f):
    A = nounce('A',IntSort()) ;  B = nounce('B',IntSort())
    Q = And(substitute(f,(X,A),(Y,B)),X==A,Y==B)
    return And(Q,INV(X,Y))

def B2(f):
    return And(T(And(X==0, f)) ,F(And(X!=0, f)) )

def T(f):
    return Implies(f,And(INV(X,Y), Y == gcd(a,b)))
        
def F(f):
    return F1(F0(f))
    
def F0(f):
    return And(FT(And(Y>X, f)), FF(And(Y<=X, f)))
    
def F1(f):
    return Implies(f,INV(X,Y)) 
    
def FF(f):
    A = nounce('A', IntSort())
    return And(substitute(f,(Y,A)),X==A-Y)
    
def FT(f):
    B = nounce('B', IntSort())
    return And(substitute(f,(Y,B)),Y==B-X)
  

# para provar que 'wpc(True)' é tautologia, prova-se que a sua negação é inconsistente
true = BoolVal(True)
refutation = Not(spc(true))



In [88]:
solver = Solver()

# a semantica 
solver.add(GCD())
# a refutação
solver.add(refutation)

TypeError: 'int' object is not callable

In [ ]:
solver.check()